In [ ]:
import jsonlines
import re
import json
import os
from copy import deepcopy

In [ ]:
with jsonlines.open('xx/ori/CMeIE-V2/CMeIE-V2_dev.jsonl', 'r') as f:
    gold_datas = [data for data in f]

In [ ]:
with open('xx/ori/CMeIE-V2/53_schemas.json', 'r') as f:
    schemas = json.load(f)

In [ ]:
direction_schemas = [(item['subject_type'], item['predicate'], item['object_type']) for item in schemas if item['direction'] == '0']

In [ ]:
gold_tuples = []
for index, data in enumerate(gold_datas):
    for spo in data['spo_list']:
        gold_tuples.append((index, spo['subject'], spo['subject_type'], spo['predicate'], spo['object']['@value'], spo['object_type']['@value']))

In [ ]:
def calc_prf(pred_tuples, gold_tuples):
    pred_tuples = set(pred_tuples)
    gold_tuples = set(gold_tuples)
    corr_tuples = pred_tuples & gold_tuples
    try:
        pre = len(corr_tuples) / len(pred_tuples)
        rec = len(corr_tuples) / len(gold_tuples)
        f1 = 2*pre*rec / (pre+rec)
    except:
        pre = 0
        rec = 0
        f1 = 0
    return pre,rec,f1

In [ ]:
def calc_prf_jiaoji(pred_tuples_1, pred_tuples_2, gold_tuples):
    pred_tuples_1 = set(pred_tuples_1)
    pred_tuples_2 = set(pred_tuples_2)
    pred_tuples = pred_tuples_1 & pred_tuples_2
    gold_tuples = set(gold_tuples)
    corr_tuples = pred_tuples & gold_tuples
    try:
        pre = len(corr_tuples) / len(pred_tuples)
        rec = len(corr_tuples) / len(gold_tuples)
        f1 = 2*pre*rec / (pre+rec)
    except:
        pre = 0
        rec = 0
        f1 = 0
    return pre,rec,f1

In [ ]:
def calc_prf_bingji(pred_tuples_1, pred_tuples_2, gold_tuples):
    pred_tuples_1 = set(pred_tuples_1)
    pred_tuples_2 = set(pred_tuples_2)
    pred_tuples = pred_tuples_1 | pred_tuples_2
    gold_tuples = set(gold_tuples)
    corr_tuples = pred_tuples & gold_tuples
    try:
        pre = len(corr_tuples) / len(pred_tuples)
        rec = len(corr_tuples) / len(gold_tuples)
        f1 = 2*pre*rec / (pre+rec)
    except:
        pre = 0
        rec = 0
        f1 = 0
    return pre,rec,f1

In [ ]:
read_files = [
'c_to_hrt_dev_CMeIE.jsonl',
'sc_to_hr_dev_CMeIE.jsonl',
'sc_to_tr_dev_CMeIE.jsonl',
]
read_dir = './pred_result'

In [ ]:
for sub_dir in os.listdir(read_dir):
    for read_file in read_files:
        with jsonlines.open(os.path.join(read_dir, sub_dir, read_file), 'r') as f:
            pred_datas = [data for data in f]
        if 'c_to_hrt' in read_file:
            pattern = r'（([^|]+?)\|\|([^|]+?)\|\|([^|]+?)\|\|([^|]+?)\|\|([^|]+?)）\s*\n'
            pred_tuples = []
            for index,data in enumerate(pred_datas):
                finds = re.findall(pattern, data['output'])
                for item in finds:
                    if (item[1], item[2], item[4]) in direction_schemas:
                        sample_1 = (index, item[0], item[1], item[2], item[3], item[4])
                        sample_2 = (index, item[3], item[1], item[2], item[0], item[4])
                        if sample_1 in gold_tuples:
                            pred_tuples.append(sample_1)
                        elif sample_2 in gold_tuples:
                            pred_tuples.append(sample_2)
                        else:
                            pred_tuples.append(sample_1)
                    else:
                        pred_tuples.append((index, item[0], item[1], item[2], item[3], item[4]))
        elif 'sc_to_hr' in read_file:
            pred_tuples = []
            for data in pred_datas:
                index = data['index']
                tr = data['tr']
                hr_list = data['hr_list']
                for hr in hr_list:
                    if (hr['subject_type'], hr['predicate'], tr['object_type']) in direction_schemas:
                        sample_1 = (index, hr['subject'], hr['subject_type'], hr['predicate'], tr['object'], tr['object_type'])
                        sample_2 = (index, tr['object'], hr['subject_type'], hr['predicate'], hr['subject'], tr['object_type'])
                        if sample_1 in gold_tuples:
                            pred_tuples.append(sample_1)
                        elif sample_2 in gold_tuples:
                            pred_tuples.append(sample_2)
                        else:
                            pred_tuples.append(sample_1)
                    else:
                        pred_tuples.append((index, hr['subject'], hr['subject_type'], hr['predicate'], tr['object'], tr['object_type']))
            pred_tuples_htc_to_r = deepcopy(pred_tuples)
        elif 'sc_to_tr' in read_file:
            pred_tuples = []
            for data in pred_datas:
                index = data['index']
                hr = data['hr']
                tr_list = data['tr_list']
                for tr in tr_list:
                    if (hr['subject_type'], tr['predicate'], tr['object_type']) in direction_schemas:
                        sample_1 = (index, hr['subject'], hr['subject_type'], tr['predicate'], tr['object'], tr['object_type'])
                        sample_2 = (index, tr['object'], hr['subject_type'], tr['predicate'], hr['subject'], tr['object_type'])
                        if sample_1 in gold_tuples:
                            pred_tuples.append(sample_1)
                        elif sample_2 in gold_tuples:
                            pred_tuples.append(sample_2)
                        else:
                            pred_tuples.append(sample_1)
                    else:
                        pred_tuples.append((index, hr['subject'], hr['subject_type'], tr['predicate'], tr['object'], tr['object_type']))
            pred_tuples_rc_to_ht = deepcopy(pred_tuples)
        pre,rec,f1 = calc_prf(pred_tuples, gold_tuples)
        print('pred_file:{}'.format(os.path.join(read_dir, sub_dir, read_file)))
        print('pre:{}\trec:{}\tf1:{}\n'.format(round(pre*100,2), round(rec*100, 2), round(f1*100, 2)))
    pre,rec,f1 = calc_prf_jiaoji(pred_tuples_htc_to_r,pred_tuples_rc_to_ht , gold_tuples)
    print('交集')
    print('pre:{}\trec:{}\tf1:{}\n'.format(round(pre*100,2), round(rec*100, 2), round(f1*100, 2)))
    pre,rec,f1 = calc_prf_bingji(pred_tuples_htc_to_r,pred_tuples_rc_to_ht , gold_tuples)
    print('并集')
    print('pre:{}\trec:{}\tf1:{}\n'.format(round(pre*100,2), round(rec*100, 2), round(f1*100, 2)))